In [ ]:
import sys
# path to the root
sys.path.append('/content/drive/MyDrive/resnet50_ft')
import torch
from data.preprocess import load_ds
from model.model_construct import Model_Construct
from train import predict_batchwise, train_model
import os
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Typical experiment on SVHN/MNIST

In [ ]:
print("----SVHN2MNIST----")
acc_mnist_ls = []
# set the number of experiments to run
for i in range(2):
  #load model
  model_10 = Model_Construct(num_classes=10).to(device)
  #load dataloaders
  mnist_trainloader, mnist_testloader, svhn_trainloader, svhn_testloader = load_ds(datasets="mnist/svhn", batch_size=64, num_workers=4, grayscale = True, normalize = False)
  # define training data
  print("--Source: svhn--")
  source_dataloader = {'train': svhn_trainloader, 'val': svhn_testloader}
  # train
  svhn_model, svhn_hist = train_model(model_10, source_dataloader, num_epochs=2, lr=0.00025, weight_decay=0, instNorm=True)
  # test
  print("--Target: mnist--")
  print("-Prediction on mnist:")
  acc_mnist, _ = predict_batchwise(svhn_model, mnist_testloader)
  acc_mnist_ls.append(acc_mnist)
# average accuracy
print("--Average accuracy--")
acc_mnist_ls = [tensor.tolist() for tensor in acc_mnist_ls]
mnist_std = np.std(acc_mnist_ls)
mnist_mean = np.mean(acc_mnist_ls)
print("Average accuracy on MNIST: {:.3f}({:.3f})".format(mnist_mean, mnist_std))


## Typical experiment on Office-31

In [ ]:
print("----Office31----")
acc_webcam_ls = []
acc_dslr_ls = []
# set the number of experiments to run
for i in range(8):
  # load model
  model_31 = Model_Construct(num_classes=31).to(device)
  # load dataloaders
  amazon_loader, amazon_halfloader, amazon_half2loader, webcam_loader, webcam_halfloader, webcam_half2loader, dslr_loader, dslr_halfloader, dslr_half2loader = load_ds(datasets="office31", batch_size=64, num_workers=4, grayscale=False, normalize=False, split_path=os.path.join("/content/drive/MyDrive/resnet50_ft", "data/splits_structure"), ds_path=os.path.join("/content/drive/MyDrive/resnet50_ft", "data/Office31"))
  print("--Source: Amazon--")
  # define training data
  source_dataloader = {'train': amazon_halfloader, 'val': amazon_half2loader}
  # train
  amazon_model, amazon_hist = train_model(model_31, source_dataloader, num_epochs=200, lr=0.001,  weight_decay=0.0001, instNorm=False)
  # test
  print("--Target: Webcam, DSLR--")
  print("Prediction on webcam:")
  acc_webcam, _ = predict_batchwise(amazon_model, webcam_loader)
  acc_webcam_ls.append(acc_webcam)
  print("Prediction on DSLR:")
  acc_dslr, _ = predict_batchwise(amazon_model, dslr_loader)
  acc_dslr_ls.append(acc_dslr)

# average accuracy
print("--Average accuracy--")
acc_webcam_ls = [tensor.tolist() for tensor in acc_webcam_ls]
webcam_std = np.std(acc_webcam_ls)
webcam_mean = np.mean(acc_webcam_ls)
print("Average accuracy on Webcam: {:.3f}({:.3f})".format(webcam_mean, webcam_std))

acc_dslr_ls = [tensor.tolist() for tensor in acc_dslr_ls]
dslr_std = np.std(acc_dslr_ls)
dslr_mean = np.mean(acc_dslr_ls)
print("Average accuracy on DSLR: {:.3f}({:.3f})".format(dslr_mean, dslr_std))
